# Linear Regression: Statsmodels vs Sci-kit Learn


## Today's Goals:

- Showcase the differences between the different implementations of ordinary least squares regression

### First: Set Up

In [1]:
# Basic imports
import numpy as np
import pandas as pd
# Data visualizations
import matplotlib.pyplot as plt
import seaborn as sns
# Pre-Processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

Credit data from https://www.kaggle.com/avikpaul4u/credit-card-balance

Target: `Balance`

In [2]:
# Data
df = pd.read_csv('data/Credit.csv', 
                 usecols=['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Balance'])

In [3]:
df.head()

,Income,Limit,Rating,Cards,Age,Balance
0,14.891,3606,283,2,34,333
1,106.025,6645,483,3,82,903
2,104.593,7075,514,4,71,580
3,148.924,9504,681,3,36,964
4,55.882,4897,357,2,68,331


In [4]:
df.describe()

,Income,Limit,Rating,Cards,Age,Balance
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,45.218885,4735.600000,354.940000,2.957500,55.667500,520.015000
std,35.244273,2308.198848,154.724143,1.371275,17.249807,459.758877
min,10.354000,855.000000,93.000000,1.000000,23.000000,0.000000
25%,21.007250,3088.000000,247.250000,2.000000,41.750000,68.750000
50%,33.115500,4622.500000,344.000000,3.000000,56.000000,459.500000
75%,57.470750,5872.750000,437.250000,4.000000,70.000000,863.000000
max,186.634000,13913.000000,982.000000,9.000000,98.000000,1999.000000


In [5]:
# Let's define our X and y
X = df.drop(columns='Balance')
y = df['Balance']

In [6]:
# Train test split here!
# Set test_size = .33
# Set random_state = 42

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [7]:
# Time to scale!
# Instantiate a new scaler
scaler = StandardScaler()

# Learn the pattern from the training data
scaler.fit(X_train)

# Apply the pattern to the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Let's turn these into dataframes
X_train_scaled = pd.DataFrame(X_train_scaled,
                              columns=X_train.columns,
                              index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled,
                             columns=X_test.columns,
                             index=X_test.index)

X_train_scaled.head()

,Income,Limit,Rating,Cards,Age
258,-0.101043,-0.938725,-0.901892,-0.669798,-1.258188
177,-0.655066,-0.344896,-0.416992,-0.669798,-1.377254
119,-0.636629,-1.425546,-1.393090,0.025961,1.420793
194,-0.421264,-1.125248,-1.116004,-0.669798,1.301727
229,0.730326,1.173512,1.188847,0.025961,1.123129


## Statsmodels' `ols`

Aka the formula version

In [8]:
# Import
from statsmodels.formula.api import ols

In [9]:
# For this version, we need to create a train_df and test_df
# This is easier because we made sure our scaled X data is a df
train_df_scaled = pd.concat([X_train_scaled, y_train], axis=1)
test_df_scaled = pd.concat([X_test_scaled, y_test], axis=1)

train_df_scaled.head()

,Income,Limit,Rating,Cards,Age,Balance
258,-0.101043,-0.938725,-0.901892,-0.669798,-1.258188,0
177,-0.655066,-0.344896,-0.416992,-0.669798,-1.377254,384
119,-0.636629,-1.425546,-1.393090,0.025961,1.420793,0
194,-0.421264,-1.125248,-1.116004,-0.669798,1.301727,0
229,0.730326,1.173512,1.188847,0.025961,1.123129,1058


In [10]:
# Now define our formula - all X variabels against y
formula = 'Balance ~ Income + Limit + Rating + Cards + Age'

# or can do:
# formula = 'Balance ~ ' + ' + '.join(X_train.columns)
formula

'Balance ~ Income + Limit + Rating + Cards + Age'

In [11]:
# Set up and fit your model
model_ols = ols(formula=formula, data=train_df_scaled).fit()

In [12]:
# Check your results!
model_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Balance   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     410.8
Date:                Mon, 18 Sep 2023   Prob (F-statistic):          9.85e-122
Time:                        12:18:28   Log-Likelihood:                -1738.6
No. Observations:                 268   AIC:                             3489.
Df Residuals:                     262   BIC:                             3511.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    539.6343      9.817     54.967      0.000     520.303     558.965
Income      -245.3279     16.442    -14.921      0.000    -277.703    -212.953
Limit        475.8959    147.989      3.216      0.001     184.498     767.294
Rating       135.5443    149.144      0.909      0.364    -158.129     429.218
Cards         31.1953     11.889      2.624      0.009       7.785      54.605
Age          -15.3400      9.989     -1.536      0.126     -35.008       4.328
==============================================================================
Omnibus:                       62.981   Durbin-Watson:                   2.062
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              104.460
Skew:                           1.330   Prob(JB):                     2.07e-23
Kurtosis:                       4.509   Cond. No.                         35.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Statsmodels' `OLS`

Aka X vs y version 

In [13]:
# Import
import statsmodels.api as sm

In [14]:
# Now we'll use our X_train_scaled and y_train!
# Note the add constant
model_OLS = sm.OLS(endog=y_train, exog=sm.add_constant(X_train_scaled)).fit()

In [15]:
# Check your results!
model_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Balance   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     410.8
Date:                Mon, 18 Sep 2023   Prob (F-statistic):          9.85e-122
Time:                        12:18:29   Log-Likelihood:                -1738.6
No. Observations:                 268   AIC:                             3489.
Df Residuals:                     262   BIC:                             3511.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        539.6343      9.817     54.967      0.000     520.303     558.965
Income      -245.3279     16.442    -14.921      0.000    -277.703    -212.953
Limit        475.8959    147.989      3.216      0.001     184.498     767.294
Rating       135.5443    149.144      0.909      0.364    -158.129     429.218
Cards         31.1953     11.889      2.624      0.009       7.785      54.605
Age          -15.3400      9.989     -1.536      0.126     -35.008       4.328
==============================================================================
Omnibus:                       62.981   Durbin-Watson:                   2.062
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              104.460
Skew:                           1.330   Prob(JB):                     2.07e-23
Kurtosis:                       4.509   Cond. No.                         35.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## And Now - SKLearn!

Aka the no-summary version

In [16]:
# Import
from sklearn.linear_model import LinearRegression

In [17]:
# Instantiate our model
model_sk = LinearRegression()

In [18]:
# Fit our model
model_sk.fit(X_train_scaled, y_train)

LinearRegression()

In [19]:
# Get our R2 score
model_sk.score(X_train_scaled, y_train)

0.886882050887701

In [20]:
# Can also use:
train_preds = model_sk.predict(X_train_scaled)

r2_score(y_train, train_preds)

0.886882050887701

In [21]:
# Check our coefficients
model_sk.coef_

array([-245.32789557,  475.89588253,  135.54433337,   31.19526402,
        -15.33999623])

In [22]:
# Add the column names to look at
dict(zip(X_train.columns, model_sk.coef_))

{'Income': -245.32789556682047,
 'Limit': 475.8958825293785,
 'Rating': 135.54433336954847,
 'Cards': 31.195264023910084,
 'Age': -15.339996226797782}

In [23]:
mean_absolute_error(y_train, train_preds)

118.44162489191918

In [24]:
mean_squared_error(y_train, train_preds)

25251.722515767826